# Coleta de Notícias — Fintech de Investimentos

```bash
pip install pandas requests beautifulsoup4 lxml html5lib duckdb tqdm
```


## Importações e Configurações

In [5]:
import os, time, re
from datetime import datetime
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from tqdm import tqdm

os.makedirs('data/raw', exist_ok=True)
os.makedirs('data/processed', exist_ok=True)

HEADERS = {
    "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                   "(KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36 OPR/104.0.0.0"),
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Language": "pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7"
}

def fetch(url, params=None, sleep=1.0):
    r = requests.get(url, headers=HEADERS, params=params, timeout=30)
    time.sleep(sleep)
    r.raise_for_status()
    return r
print('Ambiente pronto.')


✅ Ambiente pronto.


## Funções Utilitárias

In [1]:
def clean_text(txt: str) -> str:
    if not txt: return ''
    return re.sub(r'\s+', ' ', str(txt)).strip()

def to_iso_from_any(s: str):
    if not s: return None
    s = str(s)

    try:
        from email.utils import parsedate_to_datetime
        return parsedate_to_datetime(s).date().isoformat()
    except Exception:
        pass
    # tenta yyyy-mm-dd
    m = re.search(r'(\d{4}-\d{2}-\d{2})', s)
    if m: return m.group(1)
    # tenta dd/mm/yyyy
    try:
        from datetime import datetime
        return datetime.strptime(s, '%d/%m/%Y').date().isoformat()
    except Exception:
        return None


## InfoMoney — HTML (últimas notícias)

In [7]:
def coleta_infomoney_html(pages=2):
    base = "https://www.infomoney.com.br/ultimas-noticias/"
    itens = []
    for p in range(1, pages+1):
        url = base if p == 1 else f"{base}page/{p}/"
        try:
            resp = fetch(url)
            soup = BeautifulSoup(resp.text, 'lxml')
        except Exception as e:
            print(f'Falha ao acessar {url}:', e)
            continue
        for art in soup.select('article, div.card, li'):
            a = art.find('a', href=True)
            if not a: continue
            titulo = clean_text(a.get_text())
            link = urljoin(base, a['href'])
            if not link.startswith('https://www.infomoney.com.br'): continue
            if len(titulo) < 20: continue
            ttag = art.find('time')
            dt = (ttag.get('datetime') or ttag.get_text(strip=True)) if ttag else None
            itens.append({'fonte':'InfoMoney','titulo':titulo,'url':link,'data_publicacao_raw':dt})
    return pd.DataFrame(itens)

df_im = coleta_infomoney_html(pages=2)
print('InfoMoney (HTML) itens:', len(df_im))


InfoMoney (HTML) itens: 20


## Coleta via RSS — SeuDinheiro, InfoMoney, Exame Invest

In [ ]:
def coleta_rss(url_feed: str, fonte: str, max_itens=80):
    try:
        resp = fetch(url_feed)
        soup = BeautifulSoup(resp.content, 'xml')  # parser XML
    except Exception as e:
        print(f'Falha no RSS {fonte}:', e)
        return pd.DataFrame()
    itens = []
    for item in soup.find_all(['item','entry'])[:max_itens]:
        titulo = clean_text(item.title.get_text() if item.title else None)
        link = None
        if item.link:
            link = item.link.get('href') or item.link.get_text()
        elif item.find('guid'):
            link = item.find('guid').get_text()
        pub = None
        for tag in ['pubDate','published','updated','dc:date']:
            t = item.find(tag)
            if t:
                pub = clean_text(t.get_text())
                break
        if not titulo or not link: continue
        itens.append({'fonte':fonte,'titulo':titulo,'url':link,'data_publicacao_raw':pub})
    return pd.DataFrame(itens)

rss_sd   = 'https://www.seudinheiro.com/feed/'
rss_im   = 'https://www.infomoney.com.br/feed/'
rss_ex   = 'https://exame.com/feed/'

df_sd = coleta_rss(rss_sd, 'SeuDinheiro')
df_im_rss = coleta_rss(rss_im, 'InfoMoney')
df_ex = coleta_rss(rss_ex, 'ExameInvest')

print('SeuDinheiro (RSS) itens:', len(df_sd))
print('InfoMoney (RSS) itens:', len(df_im_rss))
print('Exame Invest (RSS) itens:', len(df_ex))


## Consolidação, Deduplicação e Normalização

In [ ]:
from datetime import datetime, timezone

df_all = pd.concat([df_im, df_im_rss, df_sd, df_ex], ignore_index=True)
df_all['titulo_norm'] = df_all['titulo'].str.lower().str.replace(r'\s+', ' ', regex=True).str.strip()
df_all = df_all.drop_duplicates(subset=['url']).drop_duplicates(subset=['titulo_norm']).drop(columns=['titulo_norm'])
df_all['data_publicacao'] = df_all['data_publicacao_raw'].apply(to_iso_from_any)
df_all['coletado_em'] = datetime.now(timezone.utc).isoformat(timespec='seconds')
df_all = df_all[['fonte','titulo','url','data_publicacao','coletado_em']]
print('Total após limpeza:', len(df_all))
df_all.head(10)


## Salvando Dataset cru (raw)

In [ ]:
import json

hoje = datetime.now().strftime('%Y%m%d_%H%M%S')
csv_path = f'data/raw/noticias_{hoje}.csv'
jsonl_path = f'data/raw/noticias_{hoje}.jsonl'

df_all.to_csv(csv_path, index=False, encoding='utf-8')

with open(jsonl_path, 'w', encoding='utf-8') as f:
    for _, row in df_all.iterrows():
        f.write(json.dumps(row.to_dict(), ensure_ascii=False) + '\n')

print('Arquivos salvos:')
print(' -', csv_path)
print(' -', jsonl_path)


## Inserção no DuckDB

In [20]:
import duckdb
conn = duckdb.connect('banco/banco_analitico.duckdb')
conn.execute('''
CREATE TABLE IF NOT EXISTS noticias (
    fonte           VARCHAR,
    titulo          VARCHAR,
    url             VARCHAR,
    data_publicacao DATE,
    coletado_em     TIMESTAMP
)
''')
conn.register('tmp_df', df_all)
conn.execute("""
INSERT INTO noticias
SELECT fonte, titulo, url,
       CASE WHEN data_publicacao IS NULL OR data_publicacao = '' THEN NULL
            ELSE CAST(data_publicacao AS DATE) END,
       CAST(coletado_em AS TIMESTAMPTZ)
FROM tmp_df
""")
print('Total em noticias:', conn.execute('SELECT COUNT(*) FROM noticias').fetchone()[0])
conn.close()


Total em noticias: 55


In [ ]:
import duckdb

conn = duckdb.connect('banco/banco_analitico.duckdb')
df = conn.execute("SELECT * FROM noticias").fetchdf()
df.head(10)